In [1]:
import pandas as pd
import re
import yaml
import sqlparse
import os
import pandas as pd
import requests


#### Extract repo elements

In [2]:
def extract_owner_and_repo(github_url):
    try:
        # Remove the base URL and split the rest
        parts = github_url.replace("https://github.com/", "").split("/")
        # Validate structure
        if len(parts) >= 2:
            owner = parts[0]
            repo = parts[1]
            return owner, repo
        else:
            raise ValueError("Invalid GitHub URL structure.")
    except Exception as e:
        print(f"Error: {e}")
        return None, None

def list_local_repo_structure(repo_path):
    paths = []
    for root, dirs, files in os.walk(repo_path):
        rel_dir = os.path.relpath(root, repo_path)
        if rel_dir == '.':
            rel_dir = ''
        if rel_dir:
            paths.append(rel_dir + '/')
        for f in files:
            file_path = f"{rel_dir}/{f}" if rel_dir else f
            paths.append(file_path)
    return paths

def list_online_repo_structure(owner, repo):
    url = f"https://api.github.com/repos/{owner}/{repo}/contents/"
    stack = [(url, '')]
    paths = []
    while stack:
        current_url, current_path = stack.pop()
        response = requests.get(current_url)
        if response.status_code == 200:
            items = response.json()
            for item in items:
                if item['type'] == 'dir':
                    paths.append(current_path + item['name'] + '/')
                    stack.append((item['url'], current_path + item['name'] + '/'))
                else:
                    paths.append(current_path + item['name'])
    return paths

In [3]:
def is_online_repo(path):
    return path.startswith("http://") or path.startswith("https://")

local_dbt_repo = ''
online_dbt_repo = 'https://github.com/dbt-labs/jaffle-shop'

# Use local repo?
if False:
    repo_path = local_dbt_repo
else:
    repo_path = online_dbt_repo

is_online = is_online_repo(repo_path)
if is_online:
    owner, repo = extract_owner_and_repo(online_dbt_repo)
    repo_elements = list_online_repo_structure(owner,repo)
else:
    repo_elements = list_local_repo_structure(local_dbt_repo)

print(repo_elements)

['.github/', '.gitignore', '.pre-commit-config.yaml', '.sqlfluff', '.sqlfluffignore', 'README.md', 'Taskfile.yml', 'analyses/', 'data-tests/', 'dbt_project.yml', 'jaffle-data/', 'macros/', 'models/', 'package-lock.yml', 'packages.yml', 'requirements.in', 'requirements.txt', 'seeds/', 'seeds/.gitkeep', 'models/marts/', 'models/staging/', 'models/staging/__sources.yml', 'models/staging/stg_customers.sql', 'models/staging/stg_customers.yml', 'models/staging/stg_locations.sql', 'models/staging/stg_locations.yml', 'models/staging/stg_order_items.sql', 'models/staging/stg_order_items.yml', 'models/staging/stg_orders.sql', 'models/staging/stg_orders.yml', 'models/staging/stg_products.sql', 'models/staging/stg_products.yml', 'models/staging/stg_supplies.sql', 'models/staging/stg_supplies.yml', 'models/marts/customers.sql', 'models/marts/customers.yml', 'models/marts/locations.sql', 'models/marts/locations.yml', 'models/marts/metricflow_time_spine.sql', 'models/marts/order_items.sql', 'models/m

#### Select dbt elements

In [4]:
dbt_extensions = ['.sql', '.yml', '.yaml', '.csv']

def select_dbt_elements_by_extension(dbt_extensions, repo_elements):
    # Filter elements with relevant extensions
    return [element for element in repo_elements if any(element.endswith(ext) for ext in dbt_extensions)]

repo_dbt_elements = select_dbt_elements_by_extension(dbt_extensions, repo_elements)
print(repo_dbt_elements)

def select_dbt_models(dbt_extensions, repo_dbt_elements):
    return [
        element for element in repo_dbt_elements
        if element.startswith('models/') and any(element.endswith(ext) for ext in dbt_extensions)
    ]

repo_dbt_models = select_dbt_models(dbt_extensions, repo_dbt_elements)
print(repo_dbt_models)

['.pre-commit-config.yaml', 'Taskfile.yml', 'dbt_project.yml', 'package-lock.yml', 'packages.yml', 'models/staging/__sources.yml', 'models/staging/stg_customers.sql', 'models/staging/stg_customers.yml', 'models/staging/stg_locations.sql', 'models/staging/stg_locations.yml', 'models/staging/stg_order_items.sql', 'models/staging/stg_order_items.yml', 'models/staging/stg_orders.sql', 'models/staging/stg_orders.yml', 'models/staging/stg_products.sql', 'models/staging/stg_products.yml', 'models/staging/stg_supplies.sql', 'models/staging/stg_supplies.yml', 'models/marts/customers.sql', 'models/marts/customers.yml', 'models/marts/locations.sql', 'models/marts/locations.yml', 'models/marts/metricflow_time_spine.sql', 'models/marts/order_items.sql', 'models/marts/order_items.yml', 'models/marts/orders.sql', 'models/marts/orders.yml', 'models/marts/products.sql', 'models/marts/products.yml', 'models/marts/supplies.sql', 'models/marts/supplies.yml', 'macros/cents_to_dollars.sql', 'macros/generate

In [5]:
dbt_config_elements = ['packages.yml', 'dbt_project.yml']

In [6]:
def generate_dbt_models_df(repo_dbt_models):
    data = []
    for path in repo_dbt_models:
        name = os.path.basename(path)
        extension = os.path.splitext(name)[1]
        data.append({'path': path, 'name': name, 'extension': extension})
    return pd.DataFrame(data)

dbt_models_df = generate_dbt_models_df(repo_dbt_models)
display(dbt_models_df)

,path,name,extension
0,models/staging/__sources.yml,__sources.yml,.yml
1,models/staging/stg_customers.sql,stg_customers.sql,.sql
2,models/staging/stg_customers.yml,stg_customers.yml,.yml
3,models/staging/stg_locations.sql,stg_locations.sql,.sql
4,models/staging/stg_locations.yml,stg_locations.yml,.yml
5,models/staging/stg_order_items.sql,stg_order_items.sql,.sql
6,models/staging/stg_order_items.yml,stg_order_items.yml,.yml
7,models/staging/stg_orders.sql,stg_orders.sql,.sql
8,models/staging/stg_orders.yml,stg_orders.yml,.yml
9,models/staging/stg_products.sql,stg_products.sql,.sql


#### Add code

In [7]:
if False:
    def get_base_url(repo_url):
        if repo_url.startswith("https://github.com"):
            parts = repo_url.replace("https://github.com/", "").split("/")
            owner, repo = parts[0], parts[1]
            return f"https://raw.githubusercontent.com/{owner}/{repo}/main"
        else:
            raise ValueError("URL not valid.")

    def extract_file_content(path, is_online = False, repo_base_url = None):
        try:
            if is_online:
                # Build complete URL
                file_url = f"{repo_base_url}/{path}" if repo_base_url else path
                response = requests.get(file_url)
                if response.status_code == 200:
                    return response.text
                else:
                    return f"Error: {response.status_code} {response.reason}"
            else:
                # Read content
                with open(path, 'r', encoding='utf-8') as file:
                    return file.read()
        except Exception as e:
            return f"Error: {e}"

    def add_code_column(df, is_online = False, repo_url = None):
        if is_online:
            repo_base_url = get_base_url(repo_url)
        else:
            repo_base_url = ''

        df['code'] = df['path'].apply(lambda path: extract_file_content(path, is_online, repo_base_url))
        return df

    dbt_models_df = add_code_column(dbt_models_df, is_online, online_dbt_repo)
    dbt_models_df.head(3)


In [ ]:
def get_base_url(repo_url):
    if repo_url.startswith("https://github.com"):
        parts = repo_url.replace("https://github.com/", "").split("/")
        owner, repo = parts[0], parts[1]
        return f"https://raw.githubusercontent.com/{owner}/{repo}/main"
    else:
        raise ValueError("URL not valid.")

def extract_file_content(path, is_online=False, repo_base_url=None):
    try:
        if is_online:
            # Build complete URL
            file_url = f"{repo_base_url}/{path}" if repo_base_url else path
            response = requests.get(file_url)
            if response.status_code == 200:
                content = response.text
            else:
                return f"Error: {response.status_code} {response.reason}"
        else:
            # Read content locally
            with open(path, 'r', encoding='utf-8') as file:
                content = file.read()

        # Process content based on file type
        if path.endswith(('.yml', '.yaml')):
            try:
                return yaml.safe_load(content)  # Parse YAML and return as dictionary
            except yaml.YAMLError as e:
                return f"Error parsing YAML: {e}"
        elif path.endswith('.sql'):
            try:
                return sqlparse.format(content, reindent=True, keyword_case='upper')  # Format SQL
            except Exception as e:
                return f"Error parsing SQL: {e}"
        else:
            return content  # Return plain text for other types

    except Exception as e:
        return f"Error: {e}"

def add_code_column(df, is_online=False, repo_url=None):
    if is_online:
        repo_base_url = get_base_url(repo_url)
    else:
        repo_base_url = ''

    # Extract content for each file and process it based on type
    df['code'] = df['path'].apply(lambda path: extract_file_content(path, is_online, repo_base_url))
    return df

dbt_models_df = add_code_column(dbt_models_df, is_online, online_dbt_repo)
dbt_models_df.head(3)


,path,name,extension,code
0,models/staging/__sources.yml,__sources.yml,.yml,"{'version': 2, 'sources': [{'name': 'ecom', 's..."
1,models/staging/stg_customers.sql,stg_customers.sql,.sql,WITH SOURCE AS\n (SELECT *\n FROM {{ source...
2,models/staging/stg_customers.yml,stg_customers.yml,.yml,"{'models': [{'name': 'stg_customers', 'descrip..."


In [9]:
dbt_models_df

,path,name,extension,code
0,models/staging/__sources.yml,__sources.yml,.yml,"{'version': 2, 'sources': [{'name': 'ecom', 's..."
1,models/staging/stg_customers.sql,stg_customers.sql,.sql,WITH SOURCE AS\n (SELECT *\n FROM {{ source...
2,models/staging/stg_customers.yml,stg_customers.yml,.yml,"{'models': [{'name': 'stg_customers', 'descrip..."
3,models/staging/stg_locations.sql,stg_locations.sql,.sql,WITH SOURCE AS\n (SELECT *\n FROM {{ source...
4,models/staging/stg_locations.yml,stg_locations.yml,.yml,"{'models': [{'name': 'stg_locations', 'descrip..."
5,models/staging/stg_order_items.sql,stg_order_items.sql,.sql,WITH SOURCE AS\n (SELECT *\n FROM {{ source...
6,models/staging/stg_order_items.yml,stg_order_items.yml,.yml,"{'models': [{'name': 'stg_order_items', 'descr..."
7,models/staging/stg_orders.sql,stg_orders.sql,.sql,WITH SOURCE AS\n (SELECT *\n FROM {{ source...
8,models/staging/stg_orders.yml,stg_orders.yml,.yml,"{'models': [{'name': 'stg_orders', 'descriptio..."
9,models/staging/stg_products.sql,stg_products.sql,.sql,WITH SOURCE AS\n (SELECT *\n FROM {{ source...


In [10]:
dbt_models_df.iloc[1].code

"WITH SOURCE AS\n  (SELECT *\n   FROM {{ source('ecom', 'raw_customers') }}), renamed AS\n  (SELECT ----------  ids\n id AS customer_id, ---------- text\n name AS customer_name\n   FROM SOURCE)\nSELECT *\nFROM renamed"

#### Add config block

In [11]:
def extract_config_block(sql_code):
    pattern = r"{{\s*config\((.*?)\)\s*}}"
    match = re.search(pattern, sql_code, re.DOTALL)
    return match.group(0) if match else None

def add_config_column(df):
    df['config'] = df.apply(
        lambda row: extract_config_block(row['code']) if row['extension'] == '.sql' else None,
        axis=1
    )
    return df

dbt_models_df = add_config_column(dbt_models_df)
dbt_models_df.head(3)

,path,name,extension,code,config
0,models/staging/__sources.yml,__sources.yml,.yml,"{'version': 2, 'sources': [{'name': 'ecom', 's...",None
1,models/staging/stg_customers.sql,stg_customers.sql,.sql,WITH SOURCE AS\n (SELECT *\n FROM {{ source...,None
2,models/staging/stg_customers.yml,stg_customers.yml,.yml,"{'models': [{'name': 'stg_customers', 'descrip...",None


In [34]:
dbt_models_df.iloc[0].code

"WITH SOURCE AS\n  (SELECT *\n   FROM {{ source('ecom', 'raw_customers') }}), renamed AS\n  (SELECT ----------  ids\n id AS customer_id, ---------- text\n name AS customer_name\n   FROM SOURCE)\nSELECT *\nFROM renamed"

In [13]:
test = """
{{
    config(
        materialized="table"
    )
}}
"""

dbt_models_df.at[0, 'config'] = test

In [14]:
def extract_materialized_value(config_text):
    if config_text:
        match = re.search(r"materialized\s*=\s*[\"']([^\"']+)[\"']", config_text)
        return match.group(1) if match else None
    return None

def check_is_snapshot(config_text):
    if config_text:
        return 'strategy' in config_text
    return False

dbt_models_df['materialized'] = dbt_models_df['config'].apply(extract_materialized_value)
dbt_models_df['is_snapshot'] = dbt_models_df['config'].apply(check_is_snapshot)
dbt_models_df['materialized'] = dbt_models_df.apply(lambda row: 'snapshot' if row['is_snapshot'] else row['materialized'] ,1)

def contains_jinja_code(code_text):
    if isinstance(code_text, str):
        return bool(re.search(r"{%|{#", code_text))
    return False

dbt_models_df['has_jinja_code'] = dbt_models_df['code'].apply(contains_jinja_code)


def categorize_model(name):
    if name.startswith("base"):
        return "base"
    elif name.startswith("stg"):
        return "stg"
    elif name.startswith("int"):
        return "int"
    elif name.startswith("test"):
        return "test"
    elif name.startswith("snap"):
        return "snap"
    elif name.startswith("__sources"):
        return "sources"
    else:
        return "other"

dbt_models_df['model_category'] = dbt_models_df['name'].apply(categorize_model)

def get_vertical(name, model_category):
    base_name = re.sub(r'\.[^.]+$', '', name)
    
    if model_category == 'sources':
        return 'sources'
    
    known_categories = ['stg', 'int']
    if model_category not in known_categories:
        # Para model_category = other u otras no conocidas, devolver base_name sin extensión
        return base_name
    
    # Para stg o int, extraer vertical antes de "__" o "."
    pattern = rf'^{re.escape(model_category)}_([a-z0-9_]+?)(?:__|\.|$)'
    match = re.search(pattern, base_name)
    return match.group(1) if match else base_name

dbt_models_df['vertical'] = dbt_models_df.apply(lambda row: get_vertical(row['name'], row['model_category']), axis=1)

In [16]:
def assign_yml_rows_to_each_model(dbt_models_df):
    dbt_models_df['yml_code'] = None

    yml_df = dbt_models_df[dbt_models_df['extension'] == '.yml'].copy()
    yml_df['delete'] = False

    for idx, row in yml_df.iterrows():
        base_name = row['name'].rsplit('.', 1)[0]

        sql_match = dbt_models_df[(dbt_models_df['name'] == base_name + '.sql')]

        if not sql_match.empty:
            dbt_models_df.at[sql_match.index[0], 'yml_code'] = row['code']
            yml_df.at[idx, 'delete'] = True
        else:
            yml_df.at[idx, 'yml_code'] = row['code']
            yml_df.at[idx, 'code'] = None

    yml_df = yml_df[~yml_df['delete']]

    dbt_models_df = dbt_models_df[dbt_models_df['extension'] != '.yml']

    yml_df = yml_df.drop(columns=['delete'])
    dbt_models_df = pd.concat([dbt_models_df, yml_df], ignore_index=True)

    return dbt_models_df

dbt_models_df = assign_yml_rows_to_each_model(dbt_models_df)


In [59]:
def extract_tests(yml_code):
    if not isinstance(yml_code, dict):
        return None

    tests_dict = {'columns': {}, 'unit_tests': []}

    # Extract tests from all models
    for model in yml_code.get('models', []):
        for column in model.get('columns', []):
            column_name = column.get('name')
            if column_name:
                # Combine 'tests' and 'data_tests' if present
                tests = column.get('tests', []) + column.get('data_tests', [])
                if tests:
                    tests_dict['columns'][column_name] = tests

    # Extract unit tests
    if 'unit_tests' in yml_code:
        unit_test_names = [test.get('name') for test in yml_code['unit_tests'] if test.get('name')]
        if unit_test_names:
            tests_dict['unit_tests'] = unit_test_names

    return tests_dict if tests_dict['columns'] or tests_dict['unit_tests'] else None

dbt_models_df['tests'] = dbt_models_df['yml_code'].apply(extract_tests)
dbt_models_df['has_tests'] = dbt_models_df['tests'].apply(lambda x: x is not None)


In [71]:
def extract_ids_from_query(code):
    if not isinstance(code, str):
        return None
    
    # Parse the SQL query
    parsed = sqlparse.parse(code)
    if not parsed:
        return None
    
    # Regular expression to find columns ending in '_id'
    id_pattern = re.compile(r'\b(\w+_id)\b')
    
    cte_ids = set()
    output_ids = set()
    
    for statement in parsed:
        # Flatten tokens to handle nested structures
        token_list = sqlparse.sql.TokenList(statement.tokens).flatten()
        inside_cte = False
        
        for token in token_list:
            # Detect CTE start (with keyword 'WITH')
            if token.ttype is sqlparse.tokens.Keyword and token.value.upper() == 'WITH':
                inside_cte = True
            
            # Detect SELECT after a WITH block ends
            if inside_cte and token.ttype is sqlparse.tokens.Keyword and token.value.upper() == 'SELECT':
                inside_cte = False
            
            if token.ttype is sqlparse.tokens.Name or token.ttype is None:
                match = id_pattern.search(token.value)
                if match:
                    if inside_cte:
                        cte_ids.add(match.group(1))
                    else:
                        output_ids.add(match.group(1))
    ids = {
        'cte_ids': list(cte_ids),
        'output_ids': list(output_ids)
    }
    return ids['output_ids'] if ids['output_ids'] != [] else None

dbt_models_df['sql_ids'] = dbt_models_df['code'].apply(extract_ids_from_query)


In [ ]:
def has_select_all_in_last_select(code):
    if not isinstance(code, str):
        return False

    parsed = sqlparse.parse(code)
    if not parsed:
        return False

    select_statements = [stmt for stmt in parsed if stmt.get_type() == 'SELECT']
    if not select_statements:
        return False
    last_select = select_statements[-1]

    for token in last_select.tokens:
        if token.ttype is sqlparse.tokens.Wildcard and token.value == '*':
            return True

    return False

dbt_models_df['has_select_all_in_last_select'] = dbt_models_df['code'].apply(has_select_all_in_last_select)


In [90]:
def has_group_by(code):
    if not isinstance(code, str):
        return False

    parsed = sqlparse.parse(code)
    if not parsed:
        return False
    return 'group by' in code.lower()


dbt_models_df['has_group_by'] = dbt_models_df['code'].apply(has_group_by)

In [99]:
def find_primary_key(tests_dict):
    if not isinstance(tests_dict, dict) or 'columns' not in tests_dict:
        return None

    for column, tests in tests_dict.get('columns', {}).items():
        # Check if the column has the required tests for a primary key
        if tests == ['not_null', 'unique'] or 'dbt_constraints.primary_key' in tests:
            return column
    
    return None

dbt_models_df['primary_key'] = dbt_models_df['tests'].apply(find_primary_key)

In [100]:
dbt_models_df

,path,name,extension,code,config,materialized,is_snapshot,has_jinja_code,model_category,vertical,yml_code,has_tests,tests,sql_ids,has_select_all_in_last_select,has_group_by,primary_key
0,models/staging/stg_customers.sql,stg_customers.sql,.sql,WITH SOURCE AS\n (SELECT *\n FROM {{ source...,None,None,False,False,stg,customers,"{'models': [{'name': 'stg_customers', 'descrip...",True,"{'columns': {'customer_id': ['not_null', 'uniq...",[customer_id],True,True,customer_id
1,models/staging/stg_locations.sql,stg_locations.sql,.sql,WITH SOURCE AS\n (SELECT *\n FROM {{ source...,None,None,False,False,stg,locations,"{'models': [{'name': 'stg_locations', 'descrip...",True,"{'columns': {'location_id': ['not_null', 'uniq...",[location_id],True,False,location_id
2,models/staging/stg_order_items.sql,stg_order_items.sql,.sql,WITH SOURCE AS\n (SELECT *\n FROM {{ source...,None,None,False,False,stg,order_items,"{'models': [{'name': 'stg_order_items', 'descr...",True,"{'columns': {'order_item_id': ['not_null', 'un...","[order_item_id, product_id, order_id]",True,False,order_item_id
3,models/staging/stg_orders.sql,stg_orders.sql,.sql,WITH SOURCE AS\n (SELECT *\n FROM {{ source...,None,None,False,False,stg,orders,"{'models': [{'name': 'stg_orders', 'descriptio...",True,"{'columns': {'order_id': ['not_null', 'unique'...","[store_id, customer_id, order_id, location_id]",True,False,order_id
4,models/staging/stg_products.sql,stg_products.sql,.sql,WITH SOURCE AS\n (SELECT *\n FROM {{ source...,None,None,False,False,stg,products,"{'models': [{'name': 'stg_products', 'descript...",True,"{'columns': {'product_id': ['not_null', 'uniqu...",[product_id],True,False,product_id
5,models/staging/stg_supplies.sql,stg_supplies.sql,.sql,WITH SOURCE AS\n (SELECT *\n FROM {{ source...,None,None,False,False,stg,supplies,"{'models': [{'name': 'stg_supplies', 'descript...",True,"{'columns': {'supply_uuid': ['not_null', 'uniq...","[product_id, supply_id]",True,False,supply_uuid
6,models/marts/customers.sql,customers.sql,.sql,WITH customers AS\n (SELECT *\n FROM {{ ref...,None,None,False,False,other,customers,"{'models': [{'name': 'customers', 'description...",True,"{'columns': {'customer_id': ['not_null', 'uniq...","[customer_id, order_id]",True,True,customer_id
7,models/marts/locations.sql,locations.sql,.sql,WITH locations AS\n (SELECT *\n FROM {{ ref...,None,None,False,False,other,locations,"{'semantic_models': [{'name': 'locations', 'de...",False,None,None,True,False,None
8,models/marts/metricflow_time_spine.sql,metricflow_time_spine.sql,.sql,-- metricflow_time_spine.sql\nWITH days AS (--...,None,None,False,False,other,metricflow_time_spine,None,False,None,None,True,False,None
9,models/marts/order_items.sql,order_items.sql,.sql,WITH order_items AS\n (SELECT *\n FROM {{ r...,None,None,False,False,other,order_items,"{'models': [{'name': 'order_items', 'columns':...",True,"{'columns': {'order_item_id': ['not_null', 'un...","[product_id, order_id]",True,True,order_item_id


In [98]:
dbt_models_df[['tests']].iloc[0].values[0]

{'columns': {'customer_id': ['not_null', 'unique']}, 'unit_tests': []}